In [8]:
from sklearn import preprocessing
def convert(data):
    number = preprocessing.LabelEncoder()
    data['Sex'] = number.fit_transform(data.Sex)
    number = preprocessing.LabelEncoder()   
    data=data.fillna(-999)
    return data

In [9]:
import pickle
import sys
import os
import pandas
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

from azureml.core import Run
# from utils import load_data

# create the outputs folder
os.makedirs('./outputs', exist_ok=True)

url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/kaggle-train.csv"
titanic = pandas.read_csv(url)

titanic = convert(titanic)

# load features and labels
X, Y = titanic[['Sex', 'Age', 'SibSp', 'Fare', 'Parch']].values, titanic['Survived'].values

# split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=0)
    
# get hold of the current run
run = Run.get_context()

# print("Regularization rate is {}".format(reg))

# train a logistic regression model on the training set
clf1 = LogisticRegression(multi_class="multinomial", max_iter=1000, solver="lbfgs", C=0.8).fit(X_train, Y_train)
print (clf1)

# evaluate the test set
accuracy = clf1.score(X_test, Y_test)
print ("Accuracy is {}".format(accuracy))

# serialize the model on disk in the special 'outputs' folder
print ("Export the model to model.pkl")
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()

LogisticRegression(C=0.8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
Accuracy is 0.7947761194029851
Export the model to model.pkl


In [10]:
url = "https://raw.githubusercontent.com/TheovanKraay/mldata/master/kaggle-test.csv"
test = pandas.read_csv(url)

In [13]:
#testpred = test_dummies[['Sex', 'Age', 'SibSp', 'Parch', 'Fare','FamilySize',  'Pclass_1', 'Pclass_2', 'Pclass_3', 'Title_Master','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer','Title_Royalty', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D','Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_U', 'Embarked_C','Embarked_Q', 'Embarked_S']]
testpred = test[['Sex', 'Age', 'SibSp', 'Fare', 'Parch']]
testpred = convert(testpred)
y_predict = clf1.predict(testpred.values)

testpred = test[['PassengerId','Sex', 'Age', 'SibSp', 'Fare', 'Parch']]
#testpred = test[['PassengerId']]

C:\Users\thvankra\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [14]:
#generate a submission file
sub = pandas.DataFrame()
sub['PassengerId'] = test['PassengerId']
sub['Survived'] = y_predict
print(sub)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
5            897         0
6            898         1
7            899         0
8            900         1
9            901         0
10           902         0
11           903         0
12           904         1
13           905         0
14           906         1
15           907         1
16           908         0
17           909         0
18           910         1
19           911         1
20           912         0
21           913         0
22           914         1
23           915         0
24           916         1
25           917         0
26           918         1
27           919         0
28           920         0
29           921         0
..           ...       ...
388         1280         0
389         1281         0
390         1282         0
391         1283         1
392         1284         0
3

In [15]:
#write the submission file
sub.to_csv('submission_lr.csv',index=False)